In [1]:
import os.path

import torch
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile

In [2]:
import sys
sys.path.append('./drive/MyDrive/Projects/MiniSoundFinder/lib/')

import classifiers

In [3]:
experiment_path = './drive/MyDrive/Projects/MiniSoundFinder/experiments/basic_convnet_01'

model = classifiers.BasicConvNet(input_channels=1, output_size=10)
weights_path = os.path.join(experiment_path, 'model.pth')
model.load_state_dict(torch.load(weights_path, map_location=torch.device("cpu")))
model.eval()

BasicConvNet(
  (features): Sequential(
    (0): Conv1d(1, 4, kernel_size=(80,), stride=(64,), bias=False)
    (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(4, 16, kernel_size=(3,), stride=(4,), bias=False)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(16, 16, kernel_size=(3,), stride=(4,), bias=False)
    (7): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): AdaptiveMaxPool1d(output_size=16)
  )
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)

In [4]:
# The model returns log-softmax, need to apply exp to it

class ModelProbabilities(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return torch.exp(self.model(x))

model_prob = ModelProbabilities(model)

In [5]:
MAX_LENGTH = 384000

sample_input = torch.distributions.uniform.Uniform(-10000, 10000).sample((1, 1, MAX_LENGTH))

In [6]:
model(sample_input)

tensor([[-645727.6250, -547960.2500, -600927.5000, -555259.0000, -149520.8594,
         -533245.5000, -775297.8750,       0.0000, -959467.5000, -474524.5938]],
       grad_fn=<LogSoftmaxBackward0>)

In [7]:
model_prob(sample_input)

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], grad_fn=<ExpBackward0>)

In [8]:
traced_script_module = torch.jit.trace(model_prob, sample_input)

In [9]:
traced_script_module_optimized = optimize_for_mobile(traced_script_module)

In [10]:
output_path = './drive/MyDrive/Projects/MiniSoundFinder/exports/'


traced_script_module_optimized._save_for_lite_interpreter(
    os.path.join(output_path, 'basic_convnet.pt'))

### Inference speed

In [12]:
%%time

for i in range(1000):
    inp = torch.distributions.uniform.Uniform(-10000, 10000).sample((1, 1, MAX_LENGTH))
    out = model(inp)

CPU times: user 6.37 s, sys: 24.9 ms, total: 6.39 s
Wall time: 6.93 s


In [14]:
# time collected from mobile samples
t = [0.96, 0.3, 0.6, 0.17]
print(sum(t) / len(t))

0.5075


- Average inference time GPU: 0.006 sec/sample
- Average inference time mobile: 0.5 sec/sample
- Approximate mobile slowdown: x100